**Implement Transfer learning with InceptionResNetV2-imagenet model**

In [1]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
  api_key="Your APi kEY",
  project_name="Nutrixy_Phase_one",
  workspace="kishore-yuva",
  auto_histogram_weight_logging=True,
  auto_histogram_gradient_logging=True,
  auto_histogram_activation_logging=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/kishore-yuva/nutrixy-phase-one/094b1b25dc27405e8d369076aa9297b7



In [2]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
# Create an image data generator for image augmentation
datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

In [10]:
# Create a training generator from the training folder
train_generator = datagen.flow_from_directory(
'dataset/training',
target_size=(224, 224),
batch_size=32,
class_mode='categorical',
shuffle=True
)

# Create a validation generator from the validation folder
validation_generator = datagen.flow_from_directory(
'dataset/validation',
target_size=(224, 224),
batch_size=32,
class_mode='categorical',
shuffle=False
)

# Create a evaluation generator from the evaluation folder
evaluation_generator = datagen.flow_from_directory(
'dataset/evaluation',
target_size=(224, 224),
batch_size=32,
class_mode='categorical',
shuffle=False
)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


In [13]:
# Custom callback to log metrics to Comet
class CometCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        experiment.log_metric("accuracy", logs["accuracy"], step=epoch)
        experiment.log_metric("loss", logs["loss"], step=epoch)
        experiment.log_metric("val_accuracy", logs["val_accuracy"], step=epoch)
        experiment.log_metric("val_loss", logs["val_loss"], step=epoch)

In [6]:


# Load the pretrained model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model
base_model.trainable = False
# Add a global average pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# Add a dense layer with 11 units and softmax activation for classification
output = tf.keras.layers.Dense(11, activation='softmax')(x)
# Create the model
model = tf.keras.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:

# Train the model for 10 epochs
history = model.fit(train_generator, epochs=20, validation_data=validation_generator,callbacks=[CometCallback()])

Epoch 1/20


c:\Users\Sathish\Desktop\ImageClassification_project\FI_cls_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


309/309 ━━━━━━━━━━━━━━━━━━━━ 759s 2s/step - accuracy: 0.6082 - loss: 1.2144 - val_accuracy: 0.7516 - val_loss: 0.7512
Epoch 2/20
171/309 ━━━━━━━━━━━━━━━━━━━━ 4:15 2s/step - accuracy: 0.7689 - loss: 0.6862

COMET ERROR: Due to connectivity issues, there's an error in processing the heartbeat. The experiment's status updates might be inaccurate until the connection issues are resolved.


309/309 ━━━━━━━━━━━━━━━━━━━━ 793s 3s/step - accuracy: 0.7706 - loss: 0.6795 - val_accuracy: 0.7641 - val_loss: 0.7048
Epoch 3/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 722s 2s/step - accuracy: 0.8046 - loss: 0.5918 - val_accuracy: 0.7697 - val_loss: 0.7201
Epoch 4/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 717s 2s/step - accuracy: 0.8141 - loss: 0.5617 - val_accuracy: 0.7942 - val_loss: 0.6307
Epoch 5/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 722s 2s/step - accuracy: 0.8253 - loss: 0.5415 - val_accuracy: 0.8000 - val_loss: 0.6333
Epoch 6/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 714s 2s/step - accuracy: 0.8196 - loss: 0.5371 - val_accuracy: 0.7869 - val_loss: 0.6393
Epoch 7/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 801s 3s/step - accuracy: 0.8116 - loss: 0.5496 - val_accuracy: 0.8149 - val_loss: 0.5940
Epoch 8/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 25023s 81s/step - accuracy: 0.8327 - loss: 0.5026 - val_accuracy: 0.8140 - val_loss: 0.5840
Epoch 9/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 775s 3s/step - accuracy: 0.8311 - loss: 0.4989 - val_accuracy: 0.

In [ ]:
test_loss, test_accuracy = model.evaluate(val_generator)
# Log test metrics
experiment.log_metric("test_loss", test_loss)
experiment.log_metric("test_accuracy",test_accuracy)

In [17]:
model.save('models/ImagenetReset_1000.keras', save_format='tf')
experiment.log_model("keras_model", "models")

In [ ]:
experiment.end()